# Azure OpenAI integration into APIM: function calling

Playground to try the OpenAI [function calling](https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython) feature with an Azure Functions API also managed with APIM. 

At a high level you can break down working with functions into three steps:
1. Call the chat completions API with your functions and the user’s input
2. Use the model’s response to call your API or function
3. Call the chat completions API again, including the response from your function to get a final response

![](images/architecture.png)

### Prerequisites
- [Terraform](https://learn.hashicorp.com/tutorials/terraform/install-cli) installed
- [Python 3.12 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [Azure Functions Core Tools](https://learn.microsoft.com/azure/azure-functions/functions-run-local?tabs=windows%2Cisolated-process%2Cnode-v4%2Cpython-v2%2Chttp-trigger%2Ccontainer-apps&pivots=programming-language-python#install-the-azure-functions-core-tools) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

### 1️⃣ Create deployment using Terraform

This lab uses Terraform to declaratively define all the resources that will be deployed. Change the [variables.tf](variables.tf) directly to try different configurations.

In [19]:
! $env:ARM_SUBSCRIPTION_ID=(az account show --query id -o tsv)   # if using Windows PowerShell
# ! setenv ARM_SUBSCRIPTION_ID=$(az account show --query id -o tsv) # if using macOS or Linux

! terraform init
! terraform apply -auto-approve

The filename, directory name, or volume label syntax is incorrect.


Initializing the backend...
Initializing provider plugins...
- Reusing previous version of hashicorp/azurerm from the dependency lock file
- Reusing previous version of azure/azapi from the dependency lock file
- Reusing previous version of hashicorp/random from the dependency lock file
- Using previously-installed azure/azapi v2.2.0
- Using previously-installed hashicorp/random v3.6.3
- Using previously-installed hashicorp/azurerm v4.16.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
random_string.random: Refreshing state... [id=0fj9o]
azurerm_resource_group.rg: Refreshing state... [id=/subscriptions/

The following resources will be created:

![](images/resources.png)

### 2️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [2]:
apim_resource_gateway_url = ! terraform output -raw apim_resource_gateway_url
apim_resource_gateway_url = apim_resource_gateway_url.n
print("👉🏻 APIM Resource Gateway URL: ", apim_resource_gateway_url)

apim_subscription_key = ! terraform output -raw apim_subscription_key
apim_subscription_key = apim_subscription_key.n
print("👉🏻 APIM Subscription Key: ", apim_subscription_key)

function_app_name = ! terraform output -raw function_app_name
function_app_name = function_app_name.n
print("👉🏻 Function App Name: ", function_app_name)

function_app_url = ! terraform output -raw function_app_url
function_app_url = function_app_url.n
print("👉🏻 Function App URL: ", function_app_url)

openai_api_version = "2024-10-21"
openai_model_name = "gpt-4o"
openai_deployment_name = "gpt-4o"

👉🏻 APIM Resource Gateway URL:  https://apim-genai-basicv2-0fj9o-350.azure-api.net
👉🏻 APIM Subscription Key:  d1c7c2c42d004813b2849436657ba0b4
👉🏻 Function App Name:  function-0fj9o-350
👉🏻 Function App URL:  function-0fj9o-350.azurewebsites.net


### 3️⃣ Deploy the function

Deploy the local function project to the function app resource previously created.

In [14]:
! func azure functionapp publish {function_app_name}

Local python version '3.12.8' is different from the version expected for your deployed Function App. This may result in 'ModuleNotFound' errors in Azure Functions. Please create a Python Function App for version 3.12 or change the virtual environment on your local machine to match 'PYTHON|3.11'.
Getting site publishing info...
[2025-01-27T09:16:21.339Z] Starting the function app deployment...
Creating archive for current directory...
Performing remote build for functions project.

Remote build in progress, please wait...
Updating submodules.
Preparing deployment for commit id 'aed2d6b9-b'.
PreDeployment: context.CleanOutputPath False
PreDeployment: context.OutputPath /home/site/wwwroot
Repository path is /tmp/zipdeploy/extracted
Running oryx build...
Command: oryx build /tmp/zipdeploy/extracted -o /tmp/build/expressbuild --platform python --platform-version 3.11 -i /tmp/8dd3eb35774a695 -p packagedir=.python_packages/lib/site-packages
Operation performed by Microsoft Oryx, https://githu

### 🧪 Test the Function API

Let's strat with testing the function API directly.

In [3]:
# import os
import json
# import datetime
import requests

request = { "location": "London", "unit": "celsius" }
url = f"https://{function_app_url}/api/weather"

response = requests.post(url, json = request)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("location: ", data.get("location"))
    print("unit: ", data.get("unit"))
    print("temperature: ", data.get("temperature"))
else:
    print(response.text)

location:  London
unit:  celsius
temperature:  22


Now, let's test the function API through API Management.

In [4]:
request = { "location": "London", "unit": "celsius" }
url = apim_resource_gateway_url + "/weather"
response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = request)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("location: ", data.get("location"))
    print("unit: ", data.get("unit"))
    print("temperature: ", data.get("temperature"))
else:
    print(response.text)

location:  London
unit:  celsius
temperature:  22


### 3️⃣ 🧪 Test OpenAI function calling
The following code was reused from the [Azure OpenAI documentation](https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython).

At a high level you can break down working with functions into three steps:

1) Call the chat completions API with your functions and the user’s input
2) Use the model’s response to call your API or function
3) Call the chat completions API again, including the response from your function to get a final response

- Note: run ```pip install openai``` in a terminal before executing this step.

In [5]:
from openai import AzureOpenAI
import uuid

prompt = "What's the weather like in London and its big sister cities?"

client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription_key,
    api_version=openai_api_version
)

# Example function hard coded to return the same weather
# the local function calls the API
def get_current_weather(location, unit):
    request = { "location": location, "unit": unit }
    url = apim_resource_gateway_url + "/weather"
    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = request)
    if (response.status_code == 200):
        return response.text
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

# Step 1: send the conversation and available functions to the model
messages = [{"role": "user", "content": prompt}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location, using the local temperature measuring unit.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The temperature measuring unit, e.g. celsius for London or fahrenheit for US cities",
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["location", "unit"],
            },
        },
    }
]
print("▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt\n", prompt)
response = client.chat.completions.create(
    model=openai_deployment_name,
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
if tool_calls:
    # Step 2: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    messages.append(response_message)  # extend conversation with assistant's reply
    # send the info for each function call and function response to the model
    print("▶️ Step 2: call the functions")
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        print(function_response)
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
    print("▶️ Step 3: finish with a completion to anwser the user prompt using the function response")
    second_response = client.chat.completions.create(
        model=openai_deployment_name,
        messages=messages
    )  # get a new response from the model where it can see the function response
    print(second_response.choices[0].message.content)

▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt
 What's the weather like in London and its big sister cities?
▶️ Step 2: call the functions
{"location": "London, UK", "unit": "celsius", "temperature": 20}
{"location": "New York, NY", "unit": "fahrenheit", "temperature": 20}
{"location": "Tokyo, Japan", "unit": "celsius", "temperature": 20}
{"location": "Los Angeles, CA", "unit": "fahrenheit", "temperature": 20}
{"location": "Shanghai, China", "unit": "celsius", "temperature": 20}
{"location": "Hong Kong, China", "unit": "celsius", "temperature": 20}
▶️ Step 3: finish with a completion to anwser the user prompt using the function response
Here is the current weather in London and its "big sister" cities:

- **London, UK**: 20°C
- **New York, NY**: 20°F
- **Tokyo, Japan**: 20°C
- **Los Angeles, CA**: 20°F
- **Shanghai, China**: 20°C
- **Hong Kong, China**: 20°C

If you need more details or updates, feel free to ask!
